## Extracción y transformación de datos con Spark 2.4
### Origen de datos
Los datos representan los accidentes de tráfico por día y distrito
que se han producido en la ciudad de Madrid entre 2010 y 2018

El fichero origen es un fichero csv por año tiene los datos detallados
por cada uno de los intervinientes en el accidente: Conductor, pasajero,
testito, peatón, ...

La primera parte del fichero está referida al accidente y se repite
por cada uno de los intervinientes.

### La parte del registro referida al accidente tiene los campos:
 - 'FECHA'
 - 'RANGO HORARIO'
 - 'DIA SEMANA'
 - 'DISTRITO'
 - 'LUGAR ACCIDENTE'
 - 'Nº: string'
 - 'Nº PARTE'     **Id.accidente, es el mismo para varios regs.**
 - 'CPFA Granizo'
 - 'CPFA Hielo'
 - 'CPFA Lluvia'
 - 'CPFA Niebla'
 - 'CPFA Seco'
 - 'CPFA Nieve'
 - 'CPSV Mojada'
 - 'CPSV Aceite'
 - 'CPSV Barro'
 - 'CPSV Grava Suelta'
 - 'CPSV Hielo'
 - 'CPSV Seca Y Limpia'
 - '* Nº VICTIMAS' **Total de víctimas por Nº de parte**
 - 'TIPO ACCIDENTE'
 - 'Tipo Vehiculo'
 
### La parte del registro correspondiente a cada persona:
 - 'TIPO PERSONA'
 - 'SEXO'
 - 'LESIVIDAD'
 - 'Tramo Edad'


In [10]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [11]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/datos',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)
accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

In [12]:
accidenteData.printSchema()

root
 |-- FECHA: string (nullable = true)
 |-- RANGO HORARIO: string (nullable = true)
 |-- DIA SEMANA: string (nullable = true)
 |-- DISTRITO: string (nullable = true)
 |-- LUGAR ACCIDENTE: string (nullable = true)
 |-- Nº: string (nullable = true)
 |-- Nº PARTE: string (nullable = true)
 |-- CPFA Granizo: string (nullable = true)
 |-- CPFA Hielo: string (nullable = true)
 |-- CPFA Lluvia: string (nullable = true)
 |-- CPFA Niebla: string (nullable = true)
 |-- CPFA Seco: string (nullable = true)
 |-- CPFA Nieve: string (nullable = true)
 |-- CPSV Mojada: string (nullable = true)
 |-- CPSV Aceite: string (nullable = true)
 |-- CPSV Barro: string (nullable = true)
 |-- CPSV Grava Suelta: string (nullable = true)
 |-- CPSV Hielo: string (nullable = true)
 |-- CPSV Seca Y Limpia: string (nullable = true)
 |-- * Nº VICTIMAS: string (nullable = true)
 |-- TIPO ACCIDENTE: string (nullable = true)
 |-- Tipo Vehiculo: string (nullable = true)
 |-- TIPO PERSONA: string (nullable = true)
 |-- S

In [13]:
import pyspark.sql.functions as func

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
  .filter ((func.substring(accidenteData.LESIVIDAD,1,1)=='H') | 
            (func.substring(accidenteData.LESIVIDAD,1,1)=='M') | 
            (accidenteData['TIPO PERSONA'] == 'CONDUCTOR'))\
  .select(
#        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy').alias('fecha'), \
#        accidenteData.FECHA.alias('fecha'),\
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        accidenteData["TIPO PERSONA"].alias('tipoPersona'), \
        accidenteData["SEXO"].alias('sexo'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

In [14]:
#Las columnas del Dataframe accRed quedan así
accRed.printSchema()
#Vemos si alguna columna contiene nulos
accRed.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in accRed.columns]).show()

accRed.show(n=2)


root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- tipoPersona: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- lesividad: string (nullable = true)

+-----+--------+-----------+-----------+----+---------+
|fecha|distrito|idAccidente|tipoPersona|sexo|lesividad|
+-----+--------+-----------+-----------+----+---------+
|    0|       0|          0|          0|   0|        0|
+-----+--------+-----------+-----------+----+---------+

+----------+--------------------+-----------+--------------------+------+---------+
|     fecha|            distrito|idAccidente|         tipoPersona|  sexo|lesividad|
+----------+--------------------+-----------+--------------------+------+---------+
|2018-01-01|USERA            ...|     2018/1|PEATON           ...|HOMBRE|       HG|
|2018-01-01|USERA            ...|     2018/1|           CONDUCTOR|HOMBRE|       IL|
+----------+--------------------+-----------+----------

In [15]:
# Agrupamos por fecha, distrito, accidente y generamos varias columnas contadoras 
accGrouped=accRed\
                .groupBy('fecha','distrito','idAccidente')\
                .agg(func.count(func.when(accRed.lesividad=='MT',1)).alias('muertos'),
                     func.count(func.when(accRed.lesividad=='HG',1)).alias('graves'),
                     func.count(func.when(accRed.lesividad=='HL',1)).alias('leves'),
                     func.count(func.when((accRed.sexo == 'MUJER') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condMujer'),
                     func.count(func.when((accRed.sexo == 'HOMBRE') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condHombre'),
                    )\
                .na.fill(0)\
                .sort ('fecha','distrito','idAccidente')

In [16]:
# Añadimos una columna con la indicación "Hombre" cuando los conductores son solo hombres o
# "Mujer" si son solo mujeres, en el resto de casos irá la etiqueta "Ambos"
accGrCondFila=accGrouped.select ("fecha", "distrito", "idAccidente","muertos","graves","leves", "condMujer", "condHombre")\
                .withColumn("conductorHM",
                            func.when (accGrouped.condMujer == 0, "Hombre")
                           .when (accGrouped.condHombre == 0, "Mujer")
                           .otherwise("Ambos")
                )
accGrCondFila.limit(15).toPandas()

,fecha,distrito,idAccidente,muertos,graves,leves,condMujer,condHombre,conductorHM
0,2018-01-01,ARGANZUELA,2018/25,0,0,1,0,2,Hombre
1,2018-01-01,CARABANCHEL,2018/23,0,1,0,0,2,Hombre
2,2018-01-01,CENTRO,2018/76,0,0,2,0,1,Hombre
3,2018-01-01,CHAMARTIN,2018/43,0,0,1,0,2,Hombre
4,2018-01-01,CIUDAD LINEAL,2018/12,0,1,2,0,2,Hombre
5,2018-01-01,FUENCARRAL-EL PARDO,2018/16,0,0,2,0,1,Hombre
6,2018-01-01,HORTALEZA,2018/3,0,1,0,0,1,Hombre
7,2018-01-01,MORATALAZ,2018/8,0,0,1,0,2,Hombre
8,2018-01-01,RETIRO,2018/34,0,0,2,0,2,Hombre
9,2018-01-01,SALAMANCA,2018/20,0,1,1,1,1,Ambos


In [17]:
accRes = accGrouped\
   .groupBy (accGrouped.fecha, accGrouped.distrito)\
   .agg (
            func.count(func.when(accGrouped.condMujer==0,1)).alias('condHombre'),
            func.count(func.when(accGrouped.condHombre==0,1)).alias('condMujer'),
            func.count(func.when((accGrouped.condMujer>0)&
                                 (accGrouped.condHombre>0),1)).alias('condMixto'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.muertos)).alias('muertosCondHombre'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.graves)).alias('gravesCondHombre'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.leves)).alias('levesCondHombre'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.muertos)).alias('muertosCondMujer'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.graves)).alias('gravesCondMujer'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.leves)).alias('levesCondMujer'),
            func.sum (accGrouped.muertos).alias('totalMuertos'),
            func.sum (accGrouped.graves).alias('totalGraves'),
            func.sum (accGrouped.leves).alias('totalLeves')
        )\
    .na.fill(0)\
    .orderBy (accGrouped.fecha, accGrouped.distrito)

#### Por comodidad para poder guardar todos los datos en un único fichero utilizamos la librería pandas.


In [18]:
df=accRes.toPandas()
df.to_csv('~/accidente_victimas_por_sexo_conductor.csv',sep=';')
df2=accGrCondFila.toPandas()
df2.to_csv('~/agrupadoPorAccidente_SexoConductor.csv',sep=';')
